#**1 Import**

In [ ]:
#Import generali
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
import os
import shutil
import cv2
import numpy as np
import random
from random import randint
from PIL import Image
import tensorflow as tf

In [ ]:
#Import usati per la classificazione
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression    #L1 regression, L2 regression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [ ]:
#Import usati per la classificazione con reti neurali
from tensorflow import keras
from keras import layers
from keras.layers import Dense, Flatten
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, MaxPool2D, Dropout, BatchNormalization,Activation,InputLayer,GlobalAveragePooling2D, ZeroPadding2D,Lambda
from keras import Model

In [ ]:
#Monto il contenuto di google drive su colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install opencv-python pytesseract numpy

In [ ]:
!pip install pytesseract

In [ ]:
!apt-get install tesseract-ocr

In [ ]:
!tesseract --version

In [ ]:
!apt-get install tesseract-ocr-eng
!apt-get install tesseract-ocr-ita

#**2 Classificazione con modelli di ML (senza data augmentation)**

Adesso diamo in input le features estratte sia da ResNet50 che da VGG16 a diversi modelli di classificazione e vediamo con quali di questi otteniamo l'accuracy più alta.

In [ ]:
FeatureExtractionVGG16 = pd.read_csv('/content/drive/MyDrive/ProgettoML/ProgettoDisgrafia/features_extraction/FeatureExtractionVGG16.csv')
FeatureExtractionResNet50 = pd.read_csv('/content/drive/MyDrive/ProgettoML/ProgettoDisgrafia/features_extraction/FeatureExtractionResNet50.csv')

In [ ]:
#Assegniamo alla X il dataframe con le features estratte eliminando la colonna contenente le labels(diag)

#Senza l'argomento inplace = True dobbiamo assegnare il risultato ottenuto perchè la viariabile originale non viene modificata
XResNet50 = FeatureExtractionResNet50.drop(['diag'], axis=1)

XVGG16 = FeatureExtractionVGG16.drop(['diag'], axis=1)

#la Y è uguale per entrambi i dataframe e in questo caso la prendiamo da ResNet50
Y = FeatureExtractionResNet50['diag']

##2.1 Classificazione con features estratte da ResNet50

In [ ]:
#Rimuoviamo nuovamente la colonna unnamed: 0, che compare ogni volta che andiamo a rileggere il file csv
XResNet50.drop(["Unnamed: 0"], axis=1, inplace=True)

###2.1.1 Classificazione con Logistic Regression

Definiamo una funzione che ci permette di calcolare l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).
L'accuracy media viene calcolata per i tre valori diversi dell'iperparametro C

In [ ]:
def logistic_regression_ac(model):
  Cvalues_to_accuracy_train = {1 : 0, 100 : 0, 0.01 : 0}
  Cvalues_to_accuracy_test = {1 : 0, 100 : 0, 0.01 : 0}
  RS_NUM = 100
  for i in range(RS_NUM):
   rs = random.randint(1, 100)
   X_train, X_test, y_train, y_test = train_test_split(XResNet50, Y, test_size = 20, random_state = rs)
   C_values = [1, 100, 0.01]


   for value in C_values:

     logistic_regression = model
     logistic_regression.C=value
     logistic_regression.fit(X_train, y_train)

     train_ac = logistic_regression.score(X_train,y_train)
     Cvalues_to_accuracy_train[value]  += train_ac


     test_ac = logistic_regression.score(X_test,y_test)
     Cvalues_to_accuracy_test[value]  += test_ac


  for value in C_values:
     media_train = Cvalues_to_accuracy_train[value] / RS_NUM
     media_test = Cvalues_to_accuracy_test[value] / RS_NUM
     print("Training accuracy of "+ 'logistic_regression with penalty = ' + model.penalty + " and with C={:.2f}: {:.2f}".format(value, media_train))
     print("Test accuracy of "+ 'logistic_regression with penalty = ' + model.penalty + " and with C={:.2f}: {:.2f} \n".format(value, media_test))


In [ ]:
#Calcoliamo l'accuracy media della logist_regression con l2_penalty
logistic_regression_ac(LogisticRegression())

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Training accuracy of logistic_regression with penalty = l2 and with C=1.00: 0.97
Test accuracy of logistic_regression with penalty = l2 and with C=1.00: 0.83 

Training accuracy of logistic_regression with penalty = l2 and with C=100.00: 1.00
Test accuracy of logistic_regression with penalty = l2 and with C=100.00: 0.79 

Training accuracy of logistic_regression with penalty = l2 and with C=0.01: 0.83
Test accuracy of logistic_regression with penalty = l2 and with C=0.01: 0.77 



In [ ]:
#Calcoliamo l'accuracy media della logist_regression con l1_penalty
logistic_regression_ac(LogisticRegression(penalty = 'l1', solver='liblinear'))

Training accuracy of logistic_regression with penalty = l1 and with C=1.00: 0.90
Test accuracy of logistic_regression with penalty = l1 and with C=1.00: 0.79 

Training accuracy of logistic_regression with penalty = l1 and with C=100.00: 1.00
Test accuracy of logistic_regression with penalty = l1 and with C=100.00: 0.78 

Training accuracy of logistic_regression with penalty = l1 and with C=0.01: 0.53
Test accuracy of logistic_regression with penalty = l1 and with C=0.01: 0.51 



###2.1.2 Classificazione con Decision Tree

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).

In [ ]:
RS_NUM = 100
train_ac = 0
test_ac = 0
for i in range(RS_NUM):
   rs = random.randint(1, 100)
   X_train, X_test, y_train, y_test = train_test_split(XResNet50, Y, test_size = 20, random_state = rs)

   decision_tree = DecisionTreeClassifier()
   decision_tree.fit(X_train, y_train)

   train_ac += decision_tree.score(X_train,y_train)

   test_ac += decision_tree.score(X_test,y_test)


media_train = train_ac / RS_NUM
media_test = test_ac/ RS_NUM
print("Training accuracy with decision tree classifier: {:.2f}".format(media_train))
print("Test accuracy with decision tree classifier: {:.2f}\n".format(media_test))

Training accuracy with decision tree classifier: 1.00
Test accuracy with decision tree classifier: 0.70



###2.1.3 Classificazione con Random Forest

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).
L'accuracy media viene calcolata per i tre valori diversi dell'iperparametro n_estimators

In [ ]:
Nvalues_to_accuracy_train = {100 : 0, 80 : 0, 50 : 0}
Nvalues_to_accuracy_test = {100 : 0, 80 : 0, 50 : 0}
RS_NUM = 100
for i in range(RS_NUM):
  rs = random.randint(1, 100)
  X_train, X_test, y_train, y_test = train_test_split(XResNet50, Y, test_size = 20, random_state = rs)
  n_estimators = [100, 80, 50]


  for estimator in n_estimators:

   rnd_forest = RandomForestClassifier(n_estimators= estimator)
   rnd_forest.fit(X_train, y_train)

   train_ac = rnd_forest.score(X_train,y_train)
   Nvalues_to_accuracy_train[estimator]  += train_ac


   test_ac = rnd_forest.score(X_test,y_test)
   Nvalues_to_accuracy_test[estimator]  += test_ac

for estimator in n_estimators:
  media_train = Nvalues_to_accuracy_train[estimator] / RS_NUM
  media_test = Nvalues_to_accuracy_test[estimator] / RS_NUM
  print("Training accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_train))
  print("Test accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_test))

Training accuracy of random forest classifier with n_estimators = 100 : 1.00
Test accuracy of random forest classifier with n_estimators = 100 : 0.81
Training accuracy of random forest classifier with n_estimators = 80 : 1.00
Test accuracy of random forest classifier with n_estimators = 80 : 0.80
Training accuracy of random forest classifier with n_estimators = 50 : 1.00
Test accuracy of random forest classifier with n_estimators = 50 : 0.80


###2.1.4 Classificazione con SVM

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).
L'accuracy media viene calcolata per i 4 valori diversi dell'iperparametro kernel

In [ ]:
kernel_to_accuracy_train = {'linear' : 0, 'poly' : 0, 'rbf' : 0, 'sigmoid' : 0}
kernel_to_accuracy_test = {'linear' : 0, 'poly' : 0, 'rbf' : 0, 'sigmoid' : 0}
RS_NUM = 100
for i in range(RS_NUM):
  rs = random.randint(1, 100)
  X_train, X_test, y_train, y_test = train_test_split(XResNet50, Y, test_size = 20, random_state = rs)
  kernels = ['linear', 'poly', 'rbf', 'sigmoid']


  for value in kernels:

   clf = SVC(kernel = value)
   clf.fit(X_train, y_train)

   train_ac = clf.score(X_train,y_train)
   kernel_to_accuracy_train[value]  += train_ac


   test_ac = clf.score(X_test,y_test)
   kernel_to_accuracy_test[value]  += test_ac

for value in kernels:
  media_train = kernel_to_accuracy_train[value] / RS_NUM
  media_test = kernel_to_accuracy_test[value] / RS_NUM
  print("Training accuracy of SVM classifier with kernel = {:s} : {:.2f}".format(value, media_train))
  print("Test accuracy of SVM classifier with kernel = {:s} : {:.2f}".format(value, media_test))

Training accuracy of SVM classifier with kernel = linear : 1.00
Test accuracy of SVM classifier with kernel = linear : 0.77
Training accuracy of SVM classifier with kernel = poly : 0.78
Test accuracy of SVM classifier with kernel = poly : 0.75
Training accuracy of SVM classifier with kernel = rbf : 0.75
Test accuracy of SVM classifier with kernel = rbf : 0.74
Training accuracy of SVM classifier with kernel = sigmoid : 0.55
Test accuracy of SVM classifier with kernel = sigmoid : 0.50


##2.2 Classificazione con features estratte da VGG16

In [ ]:
#Rimuoviamo nuovamente la colonna unnamed: 0, che compare ogni volta che andiamo a rileggere il file csv
XVGG16.drop(["Unnamed: 0"], axis=1, inplace=True)

###2.2.1 Classificazione con Logistic Regression

Definiamo una funzione che ci permette di calcolare l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).
L'accuracy media viene calcolata per i tre valori diversi dell'iperparametro C

In [ ]:
def logistic_regression_ac(model):
  Cvalues_to_accuracy_train = {1 : 0, 100 : 0, 0.01 : 0}
  Cvalues_to_accuracy_test = {1 : 0, 100 : 0, 0.01 : 0}
  RS_NUM = 100
  for i in range(RS_NUM):
   rs = random.randint(1, 100)
   X_train, X_test, y_train, y_test = train_test_split(XVGG16, Y, test_size = 20, random_state = rs)
   C_values = [1, 100, 0.01]


   for value in C_values:

     logistic_regression = model
     logistic_regression.C=value
     logistic_regression.fit(X_train, y_train)

     train_ac = logistic_regression.score(X_train,y_train)
     Cvalues_to_accuracy_train[value]  += train_ac


     test_ac = logistic_regression.score(X_test,y_test)
     Cvalues_to_accuracy_test[value]  += test_ac


  for value in C_values:
     media_train = Cvalues_to_accuracy_train[value] / RS_NUM
     media_test = Cvalues_to_accuracy_test[value] / RS_NUM
     print("Training accuracy of "+ 'logistic_regression with penalty = ' + model.penalty + " and with C={:.2f}: {:.2f}".format(value, media_train))
     print("Test accuracy of "+ 'logistic_regression with penalty = ' + model.penalty + " and with C={:.2f}: {:.2f} \n".format(value, media_test))


In [ ]:
#Calcoliamo l'accuracy media della logist_regression con l2_penalty
logistic_regression_ac(LogisticRegression())

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Training accuracy of logistic_regression with penalty = l2 and with C=1.00: 1.00
Test accuracy of logistic_regression with penalty = l2 and with C=1.00: 0.76 

Training accuracy of logistic_regression with penalty = l2 and with C=100.00: 1.00
Test accuracy of logistic_regression with penalty = l2 and with C=100.00: 0.75 

Training accuracy of logistic_regression with penalty = l2 and with C=0.01: 0.85
Test accuracy of logistic_regression with penalty = l2 and with C=0.01: 0.72 



In [ ]:
#Calcoliamo l'accuracy media della logist_regression con l1_penalty
logistic_regression_ac(LogisticRegression(penalty = 'l1', solver='liblinear'))

Training accuracy of logistic_regression with penalty = l1 and with C=1.00: 0.94
Test accuracy of logistic_regression with penalty = l1 and with C=1.00: 0.71 

Training accuracy of logistic_regression with penalty = l1 and with C=100.00: 1.00
Test accuracy of logistic_regression with penalty = l1 and with C=100.00: 0.72 

Training accuracy of logistic_regression with penalty = l1 and with C=0.01: 0.53
Test accuracy of logistic_regression with penalty = l1 and with C=0.01: 0.52 



###2.2.2 Classificazione con Decision Tree

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).

In [ ]:
RS_NUM = 100
train_ac = 0
test_ac = 0
for i in range(RS_NUM):
   rs = random.randint(1, 100)
   X_train, X_test, y_train, y_test = train_test_split(XVGG16, Y, test_size = 20, random_state = rs)

   decision_tree = DecisionTreeClassifier()
   decision_tree.fit(X_train, y_train)

   train_ac += decision_tree.score(X_train,y_train)

   test_ac += decision_tree.score(X_test,y_test)


media_train = train_ac / RS_NUM
media_test = test_ac/ RS_NUM
print("Training accuracy with decision tree classifier: {:.2f}".format(media_train))
print("Test accuracy with decision tree classifier: {:.2f}\n".format(media_test))

Training accuracy with decision tree classifier: 1.00
Test accuracy with decision tree classifier: 0.61



###2.2.3 Classificazione con Random Forest

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).
L'accuracy media viene calcolata per i tre valori diversi dell'iperparametro n_estimators

In [ ]:
Nvalues_to_accuracy_train = {100 : 0, 80 : 0, 50 : 0}
Nvalues_to_accuracy_test = {100 : 0, 80 : 0, 50 : 0}
RS_NUM = 100
for i in range(RS_NUM):
  rs = random.randint(1, 100)
  X_train, X_test, y_train, y_test = train_test_split(XVGG16, Y, test_size = 20, random_state = rs)
  n_estimators = [100, 80, 50]


  for estimator in n_estimators:

   rnd_forest = RandomForestClassifier(n_estimators= estimator)
   rnd_forest.fit(X_train, y_train)

   train_ac = rnd_forest.score(X_train,y_train)
   Nvalues_to_accuracy_train[estimator]  += train_ac


   test_ac = rnd_forest.score(X_test,y_test)
   Nvalues_to_accuracy_test[estimator]  += test_ac

for estimator in n_estimators:
  media_train = Nvalues_to_accuracy_train[estimator] / RS_NUM
  media_test = Nvalues_to_accuracy_test[estimator] / RS_NUM
  print("Training accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_train))
  print("Test accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_test))

Training accuracy of random forest classifier with n_estimators = 100 : 1.00
Test accuracy of random forest classifier with n_estimators = 100 : 0.67
Training accuracy of random forest classifier with n_estimators = 80 : 1.00
Test accuracy of random forest classifier with n_estimators = 80 : 0.67
Training accuracy of random forest classifier with n_estimators = 50 : 1.00
Test accuracy of random forest classifier with n_estimators = 50 : 0.66


###2.2.4 Classificazione con SVM

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).
L'accuracy media viene calcolata per i 4 valori diversi dell'iperparametro kernel

In [ ]:
kernel_to_accuracy_train = {'linear' : 0, 'poly' : 0, 'rbf' : 0, 'sigmoid' : 0}
kernel_to_accuracy_test = {'linear' : 0, 'poly' : 0, 'rbf' : 0, 'sigmoid' : 0}
RS_NUM = 100
for i in range(RS_NUM):
  rs = random.randint(1, 100)
  X_train, X_test, y_train, y_test = train_test_split(XVGG16, Y, test_size = 20, random_state = rs)
  kernels = ['linear', 'poly', 'rbf', 'sigmoid']


  for value in kernels:

   clf = SVC(kernel = value)
   clf.fit(X_train, y_train)

   train_ac = clf.score(X_train,y_train)
   kernel_to_accuracy_train[value]  += train_ac


   test_ac = clf.score(X_test,y_test)
   kernel_to_accuracy_test[value]  += test_ac

for value in kernels:
  media_train = kernel_to_accuracy_train[value] / RS_NUM
  media_test = kernel_to_accuracy_test[value] / RS_NUM
  print("Training accuracy of SVM classifier with kernel = {:s} : {:.2f}".format(value, media_train))
  print("Test accuracy of SVM classifier with kernel = {:s} : {:.2f}".format(value, media_test))

Training accuracy of SVM classifier with kernel = linear : 1.00
Test accuracy of SVM classifier with kernel = linear : 0.75
Training accuracy of SVM classifier with kernel = poly : 0.80
Test accuracy of SVM classifier with kernel = poly : 0.68
Training accuracy of SVM classifier with kernel = rbf : 0.75
Test accuracy of SVM classifier with kernel = rbf : 0.62
Training accuracy of SVM classifier with kernel = sigmoid : 0.54
Test accuracy of SVM classifier with kernel = sigmoid : 0.49


#**3 Classificazione con modelli di ML (con data augmentation)**

In [ ]:
FeatureExtractionDataAugmentationVGG16 = pd.read_csv('/content/drive/MyDrive/ProgettoML/ProgettoDisgrafia/features_extraction/FeatureExtractionDataAugmentationVGG16.csv')
FeatureExtractionDataAugmentationResNet50 = pd.read_csv('/content/drive/MyDrive/ProgettoML/ProgettoDisgrafia/features_extraction/FeatureExtractionDataAugmentationResNet50.csv')

In [ ]:
#Assegniamo alla X il dataframe con le features estratte eliminando la colonna contenente le labels(diag)

#Senza l'argomento inplace = True dobbiamo assegnare il risultato ottenuto perchè la viariabile originale non viene modificata
X_ResNet50 = FeatureExtractionDataAugmentationResNet50.drop(['diag'], axis=1)

X_VGG16 = FeatureExtractionDataAugmentationVGG16.drop(['diag'], axis=1)

#la Y è uguale per entrambi i dataframe e in questo caso la prendiamo da ResNet50
Y_da = FeatureExtractionDataAugmentationResNet50['diag']

##3.1 Classificazione con features estratte da ResNet50

In [ ]:
#Rimuoviamo nuovamente la colonna unnamed: 0, che compare ogni volta che andiamo a rileggere il file csv
X_ResNet50.drop(["Unnamed: 0"], axis=1, inplace=True)

###3.1.1 Classificazione con Logistic Regression

Definiamo una funzione che ci permette di calcolare l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).
L'accuracy media viene calcolata per i tre valori diversi dell'iperparametro C

In [ ]:
def logistic_regression_ac(model):
  Cvalues_to_accuracy_train = {1 : 0, 100 : 0, 0.01 : 0}
  Cvalues_to_accuracy_test = {1 : 0, 100 : 0, 0.01 : 0}
  RS_NUM = 100
  for i in range(RS_NUM):
   rs = random.randint(1, 100)
   X_train, X_test, y_train, y_test = train_test_split(X_ResNet50, Y_da, test_size = 20, random_state = rs)
   C_values = [1, 100, 0.01]


   for value in C_values:

     logistic_regression = model
     logistic_regression.C=value
     logistic_regression.fit(X_train, y_train)

     train_ac = logistic_regression.score(X_train,y_train)
     Cvalues_to_accuracy_train[value]  += train_ac


     test_ac = logistic_regression.score(X_test,y_test)
     Cvalues_to_accuracy_test[value]  += test_ac


  for value in C_values:
     media_train = Cvalues_to_accuracy_train[value] / RS_NUM
     media_test = Cvalues_to_accuracy_test[value] / RS_NUM
     print("Training accuracy of "+ 'logistic_regression with penalty = ' + model.penalty + " and with C={:.2f}: {:.2f}".format(value, media_train))
     print("Test accuracy of "+ 'logistic_regression with penalty = ' + model.penalty + " and with C={:.2f}: {:.2f} \n".format(value, media_test))


In [ ]:
#Calcoliamo l'accuracy media della logist_regression con l2_penalty
logistic_regression_ac(LogisticRegression())

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Training accuracy of logistic_regression with penalty = l2 and with C=1.00: 0.99
Test accuracy of logistic_regression with penalty = l2 and with C=1.00: 0.82 

Training accuracy of logistic_regression with penalty = l2 and with C=100.00: 1.00
Test accuracy of logistic_regression with penalty = l2 and with C=100.00: 0.80 

Training accuracy of logistic_regression with penalty = l2 and with C=0.01: 0.84
Test accuracy of logistic_regression with penalty = l2 and with C=0.01: 0.80 



In [ ]:
#Calcoliamo l'accuracy media della logist_regression con l1_penalty
logistic_regression_ac(LogisticRegression(penalty = 'l1', solver='liblinear'))

Training accuracy of logistic_regression with penalty = l1 and with C=1.00: 0.94
Test accuracy of logistic_regression with penalty = l1 and with C=1.00: 0.80 

Training accuracy of logistic_regression with penalty = l1 and with C=100.00: 1.00
Test accuracy of logistic_regression with penalty = l1 and with C=100.00: 0.81 

Training accuracy of logistic_regression with penalty = l1 and with C=0.01: 0.52
Test accuracy of logistic_regression with penalty = l1 and with C=0.01: 0.53 



###3.1.2 Classificazione con Decision Tree

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).

In [ ]:
RS_NUM = 100
train_ac = 0
test_ac = 0
for i in range(RS_NUM):
   rs = random.randint(1, 100)
   X_train, X_test, y_train, y_test = train_test_split(X_ResNet50, Y_da, test_size = 20, random_state = rs)

   decision_tree = DecisionTreeClassifier()
   decision_tree.fit(X_train, y_train)

   train_ac += decision_tree.score(X_train,y_train)

   test_ac += decision_tree.score(X_test,y_test)


media_train = train_ac / RS_NUM
media_test = test_ac/ RS_NUM
print("Training accuracy with decision tree classifier: {:.2f}".format(media_train))
print("Test accuracy with decision tree classifier: {:.2f}\n".format(media_test))

Training accuracy with decision tree classifier: 1.00
Test accuracy with decision tree classifier: 0.64



###3.1.3 Classificazione con Random Forest

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).
L'accuracy media viene calcolata per i tre valori diversi dell'iperparametro n_estimators

In [ ]:
Nvalues_to_accuracy_train = {100 : 0, 80 : 0, 50 : 0}
Nvalues_to_accuracy_test = {100 : 0, 80 : 0, 50 : 0}
RS_NUM = 100
for i in range(RS_NUM):
  rs = random.randint(1, 100)
  X_train, X_test, y_train, y_test = train_test_split(X_ResNet50, Y_da, test_size = 20, random_state = rs)
  n_estimators = [100, 80, 50]


  for estimator in n_estimators:

   rnd_forest = RandomForestClassifier(n_estimators= estimator)
   rnd_forest.fit(X_train, y_train)

   train_ac = rnd_forest.score(X_train,y_train)
   Nvalues_to_accuracy_train[estimator]  += train_ac


   test_ac = rnd_forest.score(X_test,y_test)
   Nvalues_to_accuracy_test[estimator]  += test_ac

for estimator in n_estimators:
  media_train = Nvalues_to_accuracy_train[estimator] / RS_NUM
  media_test = Nvalues_to_accuracy_test[estimator] / RS_NUM
  print("Training accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_train))
  print("Test accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_test))

Training accuracy of random forest classifier with n_estimators = 100 : 1.00
Test accuracy of random forest classifier with n_estimators = 100 : 0.78
Training accuracy of random forest classifier with n_estimators = 80 : 1.00
Test accuracy of random forest classifier with n_estimators = 80 : 0.78
Training accuracy of random forest classifier with n_estimators = 50 : 1.00
Test accuracy of random forest classifier with n_estimators = 50 : 0.78


###3.1.4 Classificazione con SVM

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).
L'accuracy media viene calcolata per i 4 valori diversi dell'iperparametro kernel

In [ ]:
kernel_to_accuracy_train = {'linear' : 0, 'poly' : 0, 'rbf' : 0, 'sigmoid' : 0}
kernel_to_accuracy_test = {'linear' : 0, 'poly' : 0, 'rbf' : 0, 'sigmoid' : 0}
RS_NUM = 100
for i in range(RS_NUM):
  rs = random.randint(1, 100)
  X_train, X_test, y_train, y_test = train_test_split(X_ResNet50, Y_da, test_size = 20, random_state = rs)
  kernels = ['linear', 'poly', 'rbf', 'sigmoid']


  for value in kernels:

   clf = SVC(kernel = value)
   clf.fit(X_train, y_train)

   train_ac = clf.score(X_train,y_train)
   kernel_to_accuracy_train[value]  += train_ac


   test_ac = clf.score(X_test,y_test)
   kernel_to_accuracy_test[value]  += test_ac

for value in kernels:
  media_train = kernel_to_accuracy_train[value] / RS_NUM
  media_test = kernel_to_accuracy_test[value] / RS_NUM
  print("Training accuracy of SVM classifier with kernel = {:s} : {:.2f}".format(value, media_train))
  print("Test accuracy of SVM classifier with kernel = {:s} : {:.2f}".format(value, media_test))

Training accuracy of SVM classifier with kernel = linear : 1.00
Test accuracy of SVM classifier with kernel = linear : 0.79
Training accuracy of SVM classifier with kernel = poly : 0.79
Test accuracy of SVM classifier with kernel = poly : 0.75
Training accuracy of SVM classifier with kernel = rbf : 0.77
Test accuracy of SVM classifier with kernel = rbf : 0.74
Training accuracy of SVM classifier with kernel = sigmoid : 0.72
Test accuracy of SVM classifier with kernel = sigmoid : 0.71


##3.2 Classificazione con features estratte da VGG16

In [ ]:
#Rimuoviamo nuovamente la colonna unnamed: 0, che compare ogni volta che andiamo a rileggere il file csv
X_VGG16.drop(["Unnamed: 0"], axis=1, inplace=True)

###3.2.1 Classificazione con Logistic Regression

Definiamo una funzione che ci permette di calcolare l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).
L'accuracy media viene calcolata per i tre valori diversi dell'iperparametro C

In [ ]:
def logistic_regression_ac(model):
  Cvalues_to_accuracy_train = {1 : 0, 100 : 0, 0.01 : 0}
  Cvalues_to_accuracy_test = {1 : 0, 100 : 0, 0.01 : 0}
  RS_NUM = 100
  for i in range(RS_NUM):
   rs = random.randint(1, 100)
   X_train, X_test, y_train, y_test = train_test_split(X_VGG16, Y_da, test_size = 20, random_state = rs)
   C_values = [1, 100, 0.01]


   for value in C_values:

     logistic_regression = model
     logistic_regression.C=value
     logistic_regression.fit(X_train, y_train)

     train_ac = logistic_regression.score(X_train,y_train)
     Cvalues_to_accuracy_train[value]  += train_ac


     test_ac = logistic_regression.score(X_test,y_test)
     Cvalues_to_accuracy_test[value]  += test_ac


  for value in C_values:
     media_train = Cvalues_to_accuracy_train[value] / RS_NUM
     media_test = Cvalues_to_accuracy_test[value] / RS_NUM
     print("Training accuracy of "+ 'logistic_regression with penalty = ' + model.penalty + " and with C={:.2f}: {:.2f}".format(value, media_train))
     print("Test accuracy of "+ 'logistic_regression with penalty = ' + model.penalty + " and with C={:.2f}: {:.2f} \n".format(value, media_test))


In [ ]:
#Calcoliamo l'accuracy media della logist_regression con l2_penalty
logistic_regression_ac(LogisticRegression())

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Training accuracy of logistic_regression with penalty = l2 and with C=1.00: 1.00
Test accuracy of logistic_regression with penalty = l2 and with C=1.00: 0.77 

Training accuracy of logistic_regression with penalty = l2 and with C=100.00: 1.00
Test accuracy of logistic_regression with penalty = l2 and with C=100.00: 0.77 

Training accuracy of logistic_regression with penalty = l2 and with C=0.01: 0.88
Test accuracy of logistic_regression with penalty = l2 and with C=0.01: 0.78 



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
#Calcoliamo l'accuracy media della logist_regression con l1_penalty
logistic_regression_ac(LogisticRegression(penalty = 'l1', solver='liblinear'))

Training accuracy of logistic_regression with penalty = l1 and with C=1.00: 0.98
Test accuracy of logistic_regression with penalty = l1 and with C=1.00: 0.80 

Training accuracy of logistic_regression with penalty = l1 and with C=100.00: 1.00
Test accuracy of logistic_regression with penalty = l1 and with C=100.00: 0.79 

Training accuracy of logistic_regression with penalty = l1 and with C=0.01: 0.52
Test accuracy of logistic_regression with penalty = l1 and with C=0.01: 0.53 



###3.2.2 Classificazione con Decision Tree

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).

In [ ]:
RS_NUM = 100
train_ac = 0
test_ac = 0
for i in range(RS_NUM):
   rs = random.randint(1, 100)
   X_train, X_test, y_train, y_test = train_test_split(X_VGG16, Y_da, test_size = 20, random_state = rs)

   decision_tree = DecisionTreeClassifier()
   decision_tree.fit(X_train, y_train)

   train_ac += decision_tree.score(X_train,y_train)

   test_ac += decision_tree.score(X_test,y_test)


media_train = train_ac / RS_NUM
media_test = test_ac/ RS_NUM
print("Training accuracy with decision tree classifier: {:.2f}".format(media_train))
print("Test accuracy with decision tree classifier: {:.2f}\n".format(media_test))

Training accuracy with decision tree classifier: 1.00
Test accuracy with decision tree classifier: 0.65



###3.2.3 Classificazione con Random Forest

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).
L'accuracy media viene calcolata per i tre valori diversi dell'iperparametro n_estimators

In [ ]:
Nvalues_to_accuracy_train = {100 : 0, 80 : 0, 50 : 0}
Nvalues_to_accuracy_test = {100 : 0, 80 : 0, 50 : 0}
RS_NUM = 100
for i in range(RS_NUM):
  rs = random.randint(1, 100)
  X_train, X_test, y_train, y_test = train_test_split(X_VGG16, Y_da, test_size = 20, random_state = rs)
  n_estimators = [100, 80, 50]


  for estimator in n_estimators:

   rnd_forest = RandomForestClassifier(n_estimators= estimator)
   rnd_forest.fit(X_train, y_train)

   train_ac = rnd_forest.score(X_train,y_train)
   Nvalues_to_accuracy_train[estimator]  += train_ac


   test_ac = rnd_forest.score(X_test,y_test)
   Nvalues_to_accuracy_test[estimator]  += test_ac

for estimator in n_estimators:
  media_train = Nvalues_to_accuracy_train[estimator] / RS_NUM
  media_test = Nvalues_to_accuracy_test[estimator] / RS_NUM
  print("Training accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_train))
  print("Test accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_test))

Training accuracy of random forest classifier with n_estimators = 100 : 1.00
Test accuracy of random forest classifier with n_estimators = 100 : 0.74
Training accuracy of random forest classifier with n_estimators = 80 : 1.00
Test accuracy of random forest classifier with n_estimators = 80 : 0.74
Training accuracy of random forest classifier with n_estimators = 50 : 1.00
Test accuracy of random forest classifier with n_estimators = 50 : 0.72


###3.2.4 Classificazione con SVM

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).
L'accuracy media viene calcolata per i 4 valori diversi dell'iperparametro kernel

In [ ]:
kernel_to_accuracy_train = {'linear' : 0, 'poly' : 0, 'rbf' : 0, 'sigmoid' : 0}
kernel_to_accuracy_test = {'linear' : 0, 'poly' : 0, 'rbf' : 0, 'sigmoid' : 0}
RS_NUM = 100
for i in range(RS_NUM):
  rs = random.randint(1, 100)
  X_train, X_test, y_train, y_test = train_test_split(X_VGG16, Y_da, test_size = 20, random_state = rs)
  kernels = ['linear', 'poly', 'rbf', 'sigmoid']


  for value in kernels:

   clf = SVC(kernel = value)
   clf.fit(X_train, y_train)

   train_ac = clf.score(X_train,y_train)
   kernel_to_accuracy_train[value]  += train_ac


   test_ac = clf.score(X_test,y_test)
   kernel_to_accuracy_test[value]  += test_ac

for value in kernels:
  media_train = kernel_to_accuracy_train[value] / RS_NUM
  media_test = kernel_to_accuracy_test[value] / RS_NUM
  print("Training accuracy of SVM classifier with kernel = {:s} : {:.2f}".format(value, media_train))
  print("Test accuracy of SVM classifier with kernel = {:s} : {:.2f}".format(value, media_test))

Training accuracy of SVM classifier with kernel = linear : 1.00
Test accuracy of SVM classifier with kernel = linear : 0.77
Training accuracy of SVM classifier with kernel = poly : 0.83
Test accuracy of SVM classifier with kernel = poly : 0.76
Training accuracy of SVM classifier with kernel = rbf : 0.79
Test accuracy of SVM classifier with kernel = rbf : 0.73
Training accuracy of SVM classifier with kernel = sigmoid : 0.64
Test accuracy of SVM classifier with kernel = sigmoid : 0.63


#**4 Concatenazione delle features ottenute senza data augmentation e con data augmentation**

Applichiamo questo procedimento solo a ResNet50

In [ ]:
#Otteniamo le features estratte da ResNet50 con data augmentation e senza data augmentation
FeatureExtractionResNet50 = pd.read_csv('/content/drive/MyDrive/ProgettoML/ProgettoDisgrafia/features_extraction/FeatureExtractionResNet50.csv')
FeatureExtractionDataAugmentationResNet50 = pd.read_csv('/content/drive/MyDrive/ProgettoML/ProgettoDisgrafia/features_extraction/FeatureExtractionDataAugmentationResNet50.csv')


In [ ]:
concat_dataframe = FeatureExtractionResNet50.append(FeatureExtractionDataAugmentationResNet50)
concat_dataframe.drop(["Unnamed: 0"], axis=1, inplace=True)

<ipython-input-38-5a56a74292f1>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  concat_dataframe = FeatureExtractionResNet50.append(FeatureExtractionDataAugmentationResNet50)


In [ ]:
concat_dataframe.to_csv('/content/drive/MyDrive/ProgettoML/ProgettoDisgrafia/features_extraction/ConcatFeatureResNet50.csv')

In [ ]:
X = concat_dataframe.drop(['diag'], axis=1)

Y = concat_dataframe['diag']

###4.1.1 Classificazione con Logistic Regression

Definiamo una funzione che ci permette di calcolare l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).
L'accuracy media viene calcolata per i tre valori diversi dell'iperparametro C

In [ ]:
def logistic_regression_ac(model):
  Cvalues_to_accuracy_train = {1 : 0, 100 : 0, 0.01 : 0}
  Cvalues_to_accuracy_test = {1 : 0, 100 : 0, 0.01 : 0}
  RS_NUM = 10
  for i in range(RS_NUM):
   rs = random.randint(1, 100)
   X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 20, random_state = rs)
   C_values = [1, 100, 0.01]


   for value in C_values:

     logistic_regression = model
     logistic_regression.C=value
     logistic_regression.fit(X_train, y_train)

     train_ac = logistic_regression.score(X_train,y_train)
     Cvalues_to_accuracy_train[value]  += train_ac


     test_ac = logistic_regression.score(X_test,y_test)
     Cvalues_to_accuracy_test[value]  += test_ac


  for value in C_values:
     media_train = Cvalues_to_accuracy_train[value] / RS_NUM
     media_test = Cvalues_to_accuracy_test[value] / RS_NUM
     print("Training accuracy of "+ 'logistic_regression with penalty = ' + model.penalty + " and with C={:.2f}: {:.2f}".format(value, media_train))
     print("Test accuracy of "+ 'logistic_regression with penalty = ' + model.penalty + " and with C={:.2f}: {:.2f} \n".format(value, media_test))


In [ ]:
#Calcoliamo l'accuracy media della logist_regression con l2_penalty
logistic_regression_ac(LogisticRegression())

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Training accuracy of logistic_regression with penalty = l2 and with C=1.00: 0.97
Test accuracy of logistic_regression with penalty = l2 and with C=1.00: 0.88 

Training accuracy of logistic_regression with penalty = l2 and with C=100.00: 1.00
Test accuracy of logistic_regression with penalty = l2 and with C=100.00: 0.86 

Training accuracy of logistic_regression with penalty = l2 and with C=0.01: 0.84
Test accuracy of logistic_regression with penalty = l2 and with C=0.01: 0.82 



In [ ]:
#Calcoliamo l'accuracy media della logist_regression con l1_penalty
logistic_regression_ac(LogisticRegression(penalty = 'l1', solver='liblinear'))

Training accuracy of logistic_regression with penalty = l1 and with C=1.00: 0.94
Test accuracy of logistic_regression with penalty = l1 and with C=1.00: 0.80 

Training accuracy of logistic_regression with penalty = l1 and with C=100.00: 1.00
Test accuracy of logistic_regression with penalty = l1 and with C=100.00: 0.88 

Training accuracy of logistic_regression with penalty = l1 and with C=0.01: 0.53
Test accuracy of logistic_regression with penalty = l1 and with C=0.01: 0.52 



###4.1.2 Classificazione con Decision Tree

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).

In [ ]:
RS_NUM = 10
train_ac = 0
test_ac = 0
for i in range(RS_NUM):
   rs = random.randint(1, 100)
   X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 20, random_state = rs)

   decision_tree = DecisionTreeClassifier()
   decision_tree.fit(X_train, y_train)

   train_ac += decision_tree.score(X_train,y_train)

   test_ac += decision_tree.score(X_test,y_test)


media_train = train_ac / RS_NUM
media_test = test_ac/ RS_NUM
print("Training accuracy with decision tree classifier: {:.2f}".format(media_train))
print("Test accuracy with decision tree classifier: {:.2f}\n".format(media_test))

Training accuracy with decision tree classifier: 1.00
Test accuracy with decision tree classifier: 0.73



###4.1.3 Classificazione con Random Forest

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).
L'accuracy media viene calcolata per i tre valori diversi dell'iperparametro n_estimators

In [ ]:
Nvalues_to_accuracy_train = {100 : 0, 80 : 0, 50 : 0}
Nvalues_to_accuracy_test = {100 : 0, 80 : 0, 50 : 0}
RS_NUM = 10
for i in range(RS_NUM):
  rs = random.randint(1, 100)
  X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 20, random_state = rs)
  n_estimators = [100, 80, 50]


  for estimator in n_estimators:

   rnd_forest = RandomForestClassifier(n_estimators= estimator)
   rnd_forest.fit(X_train, y_train)

   train_ac = rnd_forest.score(X_train,y_train)
   Nvalues_to_accuracy_train[estimator]  += train_ac


   test_ac = rnd_forest.score(X_test,y_test)
   Nvalues_to_accuracy_test[estimator]  += test_ac

for estimator in n_estimators:
  media_train = Nvalues_to_accuracy_train[estimator] / RS_NUM
  media_test = Nvalues_to_accuracy_test[estimator] / RS_NUM
  print("Training accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_train))
  print("Test accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_test))

Training accuracy of random forest classifier with n_estimators = 100 : 1.00
Test accuracy of random forest classifier with n_estimators = 100 : 0.81
Training accuracy of random forest classifier with n_estimators = 80 : 1.00
Test accuracy of random forest classifier with n_estimators = 80 : 0.82
Training accuracy of random forest classifier with n_estimators = 50 : 1.00
Test accuracy of random forest classifier with n_estimators = 50 : 0.82


###4.1.4 Classificazione con SVM

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).
L'accuracy media viene calcolata per i 4 valori diversi dell'iperparametro kernel

In [ ]:
kernel_to_accuracy_train = {'linear' : 0, 'poly' : 0, 'rbf' : 0, 'sigmoid' : 0}
kernel_to_accuracy_test = {'linear' : 0, 'poly' : 0, 'rbf' : 0, 'sigmoid' : 0}
RS_NUM = 10
for i in range(RS_NUM):
  rs = random.randint(1, 100)
  X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 20, random_state = rs)
  kernels = ['linear', 'poly', 'rbf', 'sigmoid']


  for value in kernels:

   clf = SVC(kernel = value)
   clf.fit(X_train, y_train)

   train_ac = clf.score(X_train,y_train)
   kernel_to_accuracy_train[value]  += train_ac


   test_ac = clf.score(X_test,y_test)
   kernel_to_accuracy_test[value]  += test_ac

for value in kernels:
  media_train = kernel_to_accuracy_train[value] / RS_NUM
  media_test = kernel_to_accuracy_test[value] / RS_NUM
  print("Training accuracy of SVM classifier with kernel = {:s} : {:.2f}".format(value, media_train))
  print("Test accuracy of SVM classifier with kernel = {:s} : {:.2f}".format(value, media_test))

Training accuracy of SVM classifier with kernel = linear : 1.00
Test accuracy of SVM classifier with kernel = linear : 0.86
Training accuracy of SVM classifier with kernel = poly : 0.81
Test accuracy of SVM classifier with kernel = poly : 0.76
Training accuracy of SVM classifier with kernel = rbf : 0.79
Test accuracy of SVM classifier with kernel = rbf : 0.76
Training accuracy of SVM classifier with kernel = sigmoid : 0.74
Test accuracy of SVM classifier with kernel = sigmoid : 0.77


#**5 Classificazione con un modello preaddestrato di InceptionV3**

In [ ]:
directory = '/content/drive/MyDrive/ProgettoML/All datasets/file/DatasetLabel1'
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory,
    labels='inferred',
    label_mode='int',
    color_mode='rgb',
    batch_size=32,
    image_size=(432, 288),
    seed=123,
    validation_split= 0.2,
    subset='training',
)

Found 360 files belonging to 2 classes.
Using 288 files for training.


In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    directory,
    labels='inferred',
    label_mode='int',
    color_mode='rgb',
    batch_size=32,
    image_size=(432, 288),
    seed=123,
    validation_split= 0.2,
    subset='validation',
)

Found 360 files belonging to 2 classes.
Using 72 files for validation.


In [ ]:
class_names = train_ds.class_names
print(class_names)

['Dysgraphic', 'No_Dysgraphic']


In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

InceptionV3 = InceptionV3(include_top=False, input_shape=(432, 288, 3), weights="imagenet")


for layer in InceptionV3.layers:
    layer.trainable = False

model = tf.keras.Sequential()

model.add(InceptionV3)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))



In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 12, 7, 2048)       21802784  
                                                                 
 global_average_pooling2d_3   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_6 (Dropout)         (None, 2048)              0         
                                                                 
 dense_8 (Dense)             (None, 256)               524544    
                                                                 
 dropout_7 (Dropout)         (None, 256)               0         
                                                                 
 dense_9 (Dense)             (None, 1)                 257       
                                                      

In [ ]:
model.compile(optimizer=Adam(learning_rate = 0.001),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
epochs=200
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/200
9/9 [==============================] - 61s 2s/step - loss: 28.7377 - accuracy: 0.5243 - val_loss: 15.7996 - val_accuracy: 0.5417
Epoch 2/200
9/9 [==============================] - 4s 295ms/step - loss: 15.1849 - accuracy: 0.5694 - val_loss: 11.9884 - val_accuracy: 0.4722
Epoch 3/200
9/9 [==============================] - 3s 248ms/step - loss: 9.1414 - accuracy: 0.5799 - val_loss: 1.8895 - val_accuracy: 0.6944
Epoch 4/200
9/9 [==============================] - 3s 248ms/step - loss: 7.6984 - accuracy: 0.5764 - val_loss: 1.2762 - val_accuracy: 0.7500
Epoch 5/200
9/9 [==============================] - 3s 247ms/step - loss: 5.7308 - accuracy: 0.5868 - val_loss: 2.2827 - val_accuracy: 0.5278
Epoch 6/200
9/9 [==============================] - 4s 338ms/step - loss: 4.6118 - accuracy: 0.6181 - val_loss: 0.5961 - val_accuracy: 0.7083
Epoch 7/200
9/9 [==============================] - 3s 240ms/step - loss: 2.9602 - accuracy: 0.5868 - val_loss: 0.6562 - val_accuracy: 0.6528
Epoch 8/200

In [ ]:
# scongeliamo i pesi dei layer del modello
InceptionV3.trainable = True


model.compile(optimizer=keras.optimizers.Adam(1e-5),
              loss='binary_crossentropy',
              metrics=[keras.metrics.BinaryAccuracy()])


history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=200
)

Epoch 1/200
9/9 [==============================] - 48s 844ms/step - loss: 0.6734 - binary_accuracy: 0.5451 - val_loss: 0.6867 - val_binary_accuracy: 0.5417
Epoch 2/200
9/9 [==============================] - 8s 781ms/step - loss: 0.6432 - binary_accuracy: 0.6215 - val_loss: 0.6901 - val_binary_accuracy: 0.5417
Epoch 3/200
9/9 [==============================] - 9s 775ms/step - loss: 0.6299 - binary_accuracy: 0.7118 - val_loss: 0.6830 - val_binary_accuracy: 0.5278
Epoch 4/200
9/9 [==============================] - 8s 706ms/step - loss: 0.5908 - binary_accuracy: 0.7847 - val_loss: 0.6816 - val_binary_accuracy: 0.5417
Epoch 5/200
9/9 [==============================] - 8s 795ms/step - loss: 0.5730 - binary_accuracy: 0.8056 - val_loss: 0.6862 - val_binary_accuracy: 0.5556
Epoch 6/200
9/9 [==============================] - 7s 713ms/step - loss: 0.5546 - binary_accuracy: 0.8333 - val_loss: 0.6924 - val_binary_accuracy: 0.5694
Epoch 7/200
9/9 [==============================] - 8s 722ms/step - lo

#**6 Classificazione con rete neurale pre-addestra sul Dataset IAM**

Il dataset IAM contiene forme di testo scritte a mano in inglese.

Il modello e i relativi pesi pre-addestrati sono stati presi da qui:
https://github.com/diegocasmo/iam_writer_recognition


In [ ]:

model = Sequential()

# Define network input shape
model.add(ZeroPadding2D((1, 1), input_shape=(432, 288, 1)))

# CNN model - Building the model suggested in paper
model.add(Conv2D(filters= 32, kernel_size =(5,5), strides= (2, 2), padding='same', name='conv1'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), name='pool1'))

model.add(Conv2D(filters= 64, kernel_size =(3, 3), strides= (1, 1), padding='same', name='conv2'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), name='pool2'))

model.add(Conv2D(filters= 128, kernel_size =(3, 3), strides= (1, 1), padding='same', name='conv3'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), name='pool3'))


model.add(Flatten())
model.add(Dropout(0.5))

model.add(Dense(512, name='dense1'))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256, name='dense2'))
model.add(Activation('relu'))
model.add(Dropout(0.5))

In [ ]:
model.load_weights('/content/drive/MyDrive/ProgettoML/model_weights.hdf5',by_name=True,skip_mismatch=True)

In [ ]:
model.trainable=False

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d (ZeroPadding  (None, 434, 290, 1)      0         
 2D)                                                             
                                                                 
 conv1 (Conv2D)              (None, 217, 145, 32)      832       
                                                                 
 activation (Activation)     (None, 217, 145, 32)      0         
                                                                 
 pool1 (MaxPooling2D)        (None, 108, 72, 32)       0         
                                                                 
 conv2 (Conv2D)              (None, 108, 72, 64)       18496     
                                                                 
 activation_1 (Activation)   (None, 108, 72, 64)       0         
                                                        

In [ ]:
new_model = Sequential()

new_model.add(model)


new_model.add(Dense(1, name='output'))
new_model.add(Activation('sigmoid'))

In [ ]:
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 256)               32075520  
                                                                 
 output (Dense)              (None, 1)                 257       
                                                                 
 activation_5 (Activation)   (None, 1)                 0         
                                                                 
Total params: 32,075,777
Trainable params: 257
Non-trainable params: 32,075,520
_________________________________________________________________


In [ ]:
directory = '/content/drive/MyDrive/ProgettoML/All datasets/file/DatasetLabel1'
train = tf.keras.utils.image_dataset_from_directory(
    directory,
    labels='inferred',
    label_mode='int',
    color_mode='grayscale',
    batch_size=32,
    image_size=(432, 288),
    seed=123,
    validation_split= 0.2,
    subset='training',
)

Found 360 files belonging to 2 classes.
Using 288 files for training.


In [ ]:
valid = tf.keras.utils.image_dataset_from_directory(
    directory,
    labels='inferred',
    label_mode='int',
    color_mode='grayscale',
    batch_size=32,
    image_size=(432, 288),
    seed=123,
    validation_split= 0.2,
    subset='validation',
)

Found 360 files belonging to 2 classes.
Using 72 files for validation.


In [ ]:
new_model.compile(optimizer=Adam(learning_rate = 0.001),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
epochs=200
history = new_model.fit(
  train,
  validation_data=valid,
  epochs=epochs
)

Epoch 1/200
9/9 [==============================] - 47s 2s/step - loss: 27.1126 - accuracy: 0.4896 - val_loss: 8.6541 - val_accuracy: 0.5417
Epoch 2/200
9/9 [==============================] - 11s 1s/step - loss: 25.3657 - accuracy: 0.4931 - val_loss: 6.2467 - val_accuracy: 0.5139
Epoch 3/200
9/9 [==============================] - 9s 933ms/step - loss: 29.0755 - accuracy: 0.4618 - val_loss: 4.7578 - val_accuracy: 0.4583
Epoch 4/200
9/9 [==============================] - 8s 818ms/step - loss: 22.1951 - accuracy: 0.5312 - val_loss: 3.7262 - val_accuracy: 0.5000
Epoch 5/200
9/9 [==============================] - 11s 1s/step - loss: 23.8400 - accuracy: 0.4931 - val_loss: 3.1826 - val_accuracy: 0.5417
Epoch 6/200
9/9 [==============================] - 10s 1s/step - loss: 25.5820 - accuracy: 0.5069 - val_loss: 3.1286 - val_accuracy: 0.5139
Epoch 7/200
9/9 [==============================] - 10s 937ms/step - loss: 23.6526 - accuracy: 0.5312 - val_loss: 3.0799 - val_accuracy: 0.5139
Epoch 8/200
9

In [ ]:
# scongeliamo i pesi dei layer del modello
model.trainable = True


new_model.compile(optimizer=keras.optimizers.Adam(1e-5),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])


history = new_model.fit(
  train,
  validation_data=valid,
  epochs=200
)

Epoch 1/200


/usr/local/lib/python3.10/dist-packages/keras/backend.py:5703: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


9/9 [==============================] - 34s 3s/step - loss: 0.8873 - binary_accuracy: 0.5486 - val_loss: 0.6807 - val_binary_accuracy: 0.5139
Epoch 2/200
9/9 [==============================] - 30s 3s/step - loss: 0.7472 - binary_accuracy: 0.5590 - val_loss: 0.6813 - val_binary_accuracy: 0.5278
Epoch 3/200
9/9 [==============================] - 31s 3s/step - loss: 0.8042 - binary_accuracy: 0.5139 - val_loss: 0.6849 - val_binary_accuracy: 0.5139
Epoch 4/200
9/9 [==============================] - 30s 3s/step - loss: 0.7971 - binary_accuracy: 0.5521 - val_loss: 0.6929 - val_binary_accuracy: 0.5000
Epoch 5/200
9/9 [==============================] - 33s 4s/step - loss: 0.6940 - binary_accuracy: 0.5938 - val_loss: 0.6912 - val_binary_accuracy: 0.5000
Epoch 6/200
9/9 [==============================] - 29s 3s/step - loss: 0.6990 - binary_accuracy: 0.6111 - val_loss: 0.6829 - val_binary_accuracy: 0.5139
Epoch 7/200
9/9 [==============================] - 31s 3s/step - loss: 0.7354 - binary_accurac

#**7 Text Detection**

In [ ]:
import cv2
import pytesseract
import numpy as np

# 1. Rilevamento delle frasi usando Tesseract OCR
def detect_text_sentences(image_path):
    # Carica l'immagine
    image = cv2.imread(image_path)
    if image is None:
        return "L'immagine non è stata caricata correttamente"

    # Esegui il rilevamento delle frasi
    d = pytesseract.image_to_data(image, output_type=pytesseract.Output.DICT)

    return d

# 2. Estrazione e salvataggio delle frasi come immagini separate
def save_text_sentences_as_images(image, data, output_directory,base_filename):
    num_boxes = len(data['text'])

    for i in range(num_boxes):
        if int(data['conf'][i]) > 40 and data['text'][i].strip():  # Ignora i box con bassa fiducia
            x, y, w, h = data['left'][i], data['top'][i], data['width'][i], data['height'][i]

            # Estrai la frase dall'immagine
            sentence_region = image[y:y+h, x:x+w]
            # Verifica se l'immagine estratta è valida (non vuota)
            if not sentence_region.size:
                continue
            # Salva la frase come immagine
            output_path = f"{output_directory}/{base_filename}_sentence_{i}.png"
            cv2.imwrite(output_path, sentence_region)
            print(f"Frase {i} salvata come immagine: {output_path}")

In [ ]:
import os

def save_image(dataset_folder,output_directory):
  # Elabora tutte le immagini nel dataset
  for filename in os.listdir(dataset_folder):
          image_path = os.path.join(dataset_folder, filename)

          # 1. Rilevamento delle frasi
          text_data = detect_text_sentences(image_path)

          # 2. Estrazione e salvataggio delle frasi come immagini separate
          image = cv2.imread(image_path)
          base_filename, _ = os.path.splitext(filename)
          save_text_sentences_as_images(image, text_data, output_directory,base_filename)


In [ ]:
# Esempio di utilizzo della funzione di text detection e salvataggio delle nuove immagini
dataset_DISG = '/content/drive/MyDrive/ProgettoML/All datasets/file/datasetLabel/Dysgraphic/'
dataset_NO_DISG = '/content/drive/MyDrive/ProgettoML/All datasets/file/datasetLabel/No_Dysgraphic/'

save_image(dataset_DISG,'/content/drive/MyDrive/ProgettoML/All datasets/file/textDetection/Dysgraphic')
save_image(dataset_NO_DISG,'/content/drive/MyDrive/ProgettoML/All datasets/file/textDetection/No_Dysgraphic')

Frase 10 salvata come immagine: /content/drive/MyDrive/ProgettoML/All datasets/file/textDetection/Dysgraphic/user00128_sentence_10.png
Frase 16 salvata come immagine: /content/drive/MyDrive/ProgettoML/All datasets/file/textDetection/Dysgraphic/user00128_sentence_16.png
Frase 18 salvata come immagine: /content/drive/MyDrive/ProgettoML/All datasets/file/textDetection/Dysgraphic/user00128_sentence_18.png
Frase 33 salvata come immagine: /content/drive/MyDrive/ProgettoML/All datasets/file/textDetection/Dysgraphic/user00128_sentence_33.png
Frase 9 salvata come immagine: /content/drive/MyDrive/ProgettoML/All datasets/file/textDetection/Dysgraphic/user00141_sentence_9.png
Frase 14 salvata come immagine: /content/drive/MyDrive/ProgettoML/All datasets/file/textDetection/Dysgraphic/user00141_sentence_14.png
Frase 25 salvata come immagine: /content/drive/MyDrive/ProgettoML/All datasets/file/textDetection/Dysgraphic/user00141_sentence_25.png
Frase 26 salvata come immagine: /content/drive/MyDrive/Pr

In [ ]:
import os
from PIL import Image

def resize_image(image, target_height):
    aspect_ratio = target_height / float(image.height)
    target_width = int(image.width * aspect_ratio)
    resized_image = image.resize((target_width, target_height))
    return resized_image

def pad_image(image, target_width, target_height):
    pad_width = max(0, target_width - image.width)
    pad_height = max(0, target_height - image.height)
    padded_image = Image.new(image.mode, (target_width, target_height), (255, 255, 255))
    padded_image.paste(image, ((pad_width // 2), (pad_height // 2)))
    return padded_image

def process_images(input_folder, output_folder, target_height, target_width):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)

            original_image = Image.open(input_path)
            resized_image = resize_image(original_image, target_height)
            padded_image = pad_image(resized_image, target_width, target_height)
            padded_image.save(output_path)

    print("Processo completato. Immagini modificate salvate in:", output_folder)

# Percorsi delle cartelle di input e output
input_folder = '/content/drive/MyDrive/ProgettoML/All datasets/file/textDetection/Dysgraphic'
output_folder = '/content/drive/MyDrive/ProgettoML/All datasets/file/textDetection/Dysgraphic'

input_folder1 = '/content/drive/MyDrive/ProgettoML/All datasets/file/textDetection/No_Dysgraphic'
output_folder1 = '/content/drive/MyDrive/ProgettoML/All datasets/file/textDetection/No_Dysgraphic'

target_height = 75  # Sostituisci con l'altezza desiderata
target_width = 299  # Larghezza richiesta per InceptionV3

# Processo le immagini
process_images(input_folder, output_folder, target_height, target_width)
process_images(input_folder1, output_folder1, target_height, target_width)


Processo completato. Immagini modificate salvate in: /content/drive/MyDrive/ProgettoML/All datasets/file/textDetection/Dysgraphic
Processo completato. Immagini modificate salvate in: /content/drive/MyDrive/ProgettoML/All datasets/file/textDetection/No_Dysgraphic


In [ ]:
directory = '/content/drive/MyDrive/ProgettoML/All datasets/file/textDetection'
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory,
    labels='inferred',
    label_mode='int',
    color_mode='rgb',
    batch_size=32,
    image_size=(75, 299),
    seed=123,
    validation_split= 0.2,
    subset='training',
)

Found 868 files belonging to 2 classes.
Using 695 files for training.


In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    directory,
    labels='inferred',
    label_mode='int',
    color_mode='rgb',
    batch_size=32,
    image_size=(75, 299),
    seed=123,
    validation_split= 0.2,
    subset='validation',
)

Found 868 files belonging to 2 classes.
Using 173 files for validation.


In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten

InceptionV3 = InceptionV3(include_top=False, input_shape=(75, 299, 3), weights="imagenet")


for layer in InceptionV3.layers:
    layer.trainable = False

model = tf.keras.Sequential()

model.add(InceptionV3)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))


87910968/87910968 [==============================] - 0s 0us/step


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 1, 8, 2048)        21802784  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                        

In [ ]:
model.compile(optimizer=Adam(learning_rate = 0.001),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
epochs=200
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/200
22/22 [==============================] - 127s 3s/step - loss: 62.5677 - accuracy: 0.5079 - val_loss: 22.6464 - val_accuracy: 0.5145
Epoch 2/200
22/22 [==============================] - 2s 68ms/step - loss: 32.8652 - accuracy: 0.5094 - val_loss: 3.4884 - val_accuracy: 0.5202
Epoch 3/200
22/22 [==============================] - 2s 61ms/step - loss: 10.5255 - accuracy: 0.5223 - val_loss: 1.0530 - val_accuracy: 0.5087
Epoch 4/200
22/22 [==============================] - 2s 91ms/step - loss: 2.7768 - accuracy: 0.5165 - val_loss: 0.7147 - val_accuracy: 0.4971
Epoch 5/200
22/22 [==============================] - 2s 74ms/step - loss: 1.1360 - accuracy: 0.5180 - val_loss: 0.7080 - val_accuracy: 0.5087
Epoch 6/200
22/22 [==============================] - 2s 61ms/step - loss: 0.8578 - accuracy: 0.5468 - val_loss: 0.7057 - val_accuracy: 0.5145
Epoch 7/200
22/22 [==============================] - 2s 65ms/step - loss: 0.9233 - accuracy: 0.5122 - val_loss: 0.7061 - val_accuracy: 0.5087
Ep

In [ ]:
# scongeliamo i pesi dei layer del modello
InceptionV3.trainable = True


model.compile(optimizer=keras.optimizers.Adam(1e-5),
              loss='binary_crossentropy',
              metrics=[keras.metrics.BinaryAccuracy()])


history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=200
)

Epoch 1/200
22/22 [==============================] - 39s 188ms/step - loss: 0.0102 - binary_accuracy: 1.0000 - val_loss: 1.3102 - val_binary_accuracy: 0.6763
Epoch 2/200
22/22 [==============================] - 3s 128ms/step - loss: 0.0114 - binary_accuracy: 0.9986 - val_loss: 1.3465 - val_binary_accuracy: 0.6705
Epoch 3/200
22/22 [==============================] - 4s 148ms/step - loss: 0.0072 - binary_accuracy: 1.0000 - val_loss: 1.3287 - val_binary_accuracy: 0.6705
Epoch 4/200
22/22 [==============================] - 3s 136ms/step - loss: 0.0066 - binary_accuracy: 1.0000 - val_loss: 1.3822 - val_binary_accuracy: 0.6763
Epoch 5/200
22/22 [==============================] - 3s 130ms/step - loss: 0.0072 - binary_accuracy: 1.0000 - val_loss: 1.4459 - val_binary_accuracy: 0.6705
Epoch 6/200
22/22 [==============================] - 3s 130ms/step - loss: 0.0073 - binary_accuracy: 1.0000 - val_loss: 1.4394 - val_binary_accuracy: 0.6879
Epoch 7/200
22/22 [==============================] - 4s 1